In [1]:
import json
import itertools
import os
import pandas as pd
from matplotlib import pyplot as plt
from IPython.core.display import HTML
import subprocess
import numpy as np

In [2]:
os.system("g++ -ffast-math -march=native -O3 measure_linvbin.cpp")
os.system("./a.out > bvl.csv")

df = pd.read_csv("bvl.csv", header=None, names=["method", "err", "time"])
df["time"] /= 500000


from scipy.stats import linregress
x = df[df.method == "linear"]["err"]
y = df[df.method == "linear"]["time"]

lslope, lintercept, lrval, _, _ = linregress(x, y)
def lin_mod(x):
    return (lslope * x + lintercept).clip(lower=0)

assert lrval > 0.98

plt.scatter(x, y)
plt.scatter(x, lin_mod(x))

ValueError: Inputs must not be empty.

In [ ]:
import numpy as np
x = df[df.method == "binary"]["err"]
y = df[df.method == "binary"]["time"]
slope, intercept, rval, _, _ = linregress(np.log(x), y)
print(rval)
def bin_mod(x):
    return (np.log(x) * slope + intercept).clip(lower=0)

plt.scatter(x, y)
plt.scatter(x, bin_mod(x))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

bin_pred = KNeighborsRegressor(n_neighbors=2)
bin_pred.fit(df[df.method == "binary"][["err"]], df[df.method == "binary"]["time"])

lin_pred = KNeighborsRegressor(n_neighbors=2)
lin_pred.fit(df[df.method == "linear"][["err"]], df[df.method == "linear"]["time"])

In [ ]:
lin_pred.predict([[1]])
bin_pred.predict([[1]])

In [ ]:
DATA_PATH = "/home/ryan/SOSD/data/lognormal_200M_uint64"
DATA_FILE = DATA_PATH.split("/")[-1]

os.system("cd .. && cargo build --release")
RMI_PATH = "../target/release/rmi"

In [ ]:
top_only_layers = ["radix", "bradix"]
anywhere_layers = ["linear", "cubic"]
specialty_top_layers = ["histogram", "loglinear", "normal", "lognormal"]
branching_factors = [2**x for x in range(7, 22)]

In [ ]:
all_top_layers = top_only_layers + anywhere_layers

In [ ]:
# first, build a grid of the most likely configs
configs = []
nmspc_counter = 0
for top in all_top_layers:
    for bot in anywhere_layers:
        for bf in branching_factors[::3]:
            configs.append({"layers": f"{top},{bot}", "branching factor": bf, "namespace": f"nm{nmspc_counter}"})
            nmspc_counter += 1
            
# next, build a few tests to see if a speciality layer would help
for top in specialty_top_layers:
    if top == "histogram":
        for bot in anywhere_layers:
            for bf in [64, 128, 256]:
                configs.append({"layers": f"{top},{bot}", "branching factor": bf, "namespace": f"nm{nmspc_counter}"})
                nmspc_counter += 1
    else:
        # not a histogram
        for bot in anywhere_layers:
            for bf in branching_factors[::4]:
                configs.append({"layers": f"{top},{bot}", "branching factor": bf, "namespace": f"nm{nmspc_counter}"})
                nmspc_counter += 1


In [ ]:
def parallel_test_rmis(configs, threads=8, phase="", fast_top=False):
    if len(configs) < threads:
        threads = len(configs)
        
    os.system("rm *.json *.json_results")
    jobs = [[] for _ in range(threads)]
    procs = []
    
    for idx, conf in enumerate(configs):
        jobs[idx % threads].append(conf)
        
    for idx, workset in enumerate(jobs):
        fn = f"{phase}{idx}.json"
        with open(fn, "w") as f:
            json.dump({"configs": workset}, f)
        cmd = f"{RMI_PATH} {DATA_PATH} --param-grid {fn}"
        if fast_top:
            cmd += " --fast-top"
            
        procs.append(subprocess.Popen(cmd, shell=True))

    print("Spawned", threads, "processes with", [len(x) for x in jobs], "jobs each")
    
    retry = []
    for idx, proc in enumerate(procs):
        if proc.wait() != 0:
            print("Failure in RMI construction at idx", idx, "queued for retry")
            retry.append(idx)
           
    procs = []
    for idx in retry:
        fn = f"{phase}{idx}.json"
        cmd = f"{RMI_PATH} {DATA_PATH} --param-grid {fn}"
        if fast_top:
            cmd += " --fast-top"
            
        procs.append(subprocess.Popen(cmd, shell=True))
        
    for idx, proc in enumerate(procs):
        if proc.wait() != 0:
            print("Failure in retry of RMI construction at idx", idx)
            print("Perhaps lower the thread count?")
            assert False
            
    os.system("sync")
    os.system("rm -rf opt/")
    os.system("mkdir opt")
    os.system("mv nm* opt/")
    data = []
    for idx, _ in enumerate(jobs):
        fn = f"{phase}{idx}.json_results"
        with open(fn, "r") as f:
            data.extend(json.load(f))

    return data
    

In [ ]:
print("Testing", len(configs), "initial configurations.")

step1_results = parallel_test_rmis(configs, phase="step1", fast_top=True)
step1_results = pd.DataFrame(step1_results)

In [ ]:
display(HTML(step1_results.to_html()))

In [ ]:
import pystache
with open("bench.cpp", "r") as f:
    template = f.read()
    
with open("to_build.cpp", "w") as f:
    f.write(pystache.render(template, 
                            {"filename": DATA_PATH, 
                             "namespaces": step1_results.namespace.tolist()}))
    
if os.system("make -j 40") != 0:
    print("Error compiling inference program!")
    
os.system("./a.out > inference.txt")
with open("inference.txt") as f:
    inference = list(int(x.strip()[:-2]) / 100000.0 for x in f)
    
step1_results["inference"] = inference

In [ ]:
step1_results["pred_time_lin"] = step1_results.inference + lin_pred.predict(step1_results[["max error"]])
step1_results["pred_time_bin"] = step1_results.inference + bin_pred.predict(step1_results[["max error"]])
step1_results["pred_time"] = step1_results[["pred_time_lin", "pred_time_bin"]].min(axis=1)

step1_results["star"] = False
step1_results.loc[step1_results["layers"] == "bradix,linear", "star"] = True

display(HTML(step1_results.sort_values("layers").to_html()))

In [ ]:
def pareto_mask(df, v1="size linear search", v2="pred_time", soft=True, ignore_star=False):
    # find Pareto efficient RMIs
    mask = []
    for idx1, el1 in df.iterrows():
        my_size = el1[v1]
        my_time = el1[v2]
        starred = el1["star"]
        for idx2, el2 in df.iterrows():
            if idx1 == idx2:
                continue

            if ((el2[v1] <= my_size) 
                and (el2[v2] <= my_time - (10 if soft else 0))):
                if (not starred) or ignore_star:
                    mask.append(False)
                    break
        else:
            mask.append(True)
    return mask

In [ ]:
mask = pareto_mask(step1_results)
pareto = step1_results[mask]
print("Found", len(pareto), "Pareto efficient models")
display(HTML(pareto.to_html()))

In [ ]:
candidate_layers = set(pareto["layers"])
next_configs = []
for candidate in candidate_layers:
    if candidate.startswith("histogram"):
        for bf in [32, 300, 512]:
            next_configs.append({"layers": candidate, "branching factor": bf,
                                 "namespace": f"nm{nmspc_counter}"})
            nmspc_counter += 1

    else:
        already_known = step1_results[step1_results.layers == candidate]["branching factor"].to_list()
        for bf in sorted(set(branching_factors) - set(already_known)):
            next_configs.append({"layers": candidate, "branching factor": bf,
                                 "namespace": f"nm{nmspc_counter}"})
            nmspc_counter += 1

In [ ]:
print("Testing", len(next_configs), "additional configurations.")

step2_results = parallel_test_rmis(next_configs, phase="step2", fast_top=True)
step2_results = pd.DataFrame(step2_results)

In [ ]:
import pystache
with open("bench.cpp", "r") as f:
    template = f.read()
    
with open("to_build.cpp", "w") as f:
    f.write(pystache.render(template, 
                            {"filename": DATA_PATH, 
                             "namespaces": step2_results.namespace.tolist()}))
    

In [ ]:
print("Compiling...")
os.system("make -j 40")

print("Executing...")
os.system("./a.out > inference.txt")
with open("inference.txt") as f:
    inference = list(int(x.strip()[:-2]) / 100000.0 for x in f)
    
step2_results["inference"] = inference

In [ ]:
step2_results["pred_time_lin"] = step2_results.inference + lin_pred.predict(step2_results[["max error"]])
step2_results["pred_time_bin"] = step2_results.inference + bin_pred.predict(step2_results[["max error"]])
step2_results["pred_time"] = step2_results[["pred_time_lin", "pred_time_bin"]].min(axis=1)

step2_results["star"] = False
step2_results.loc[step2_results["layers"] == "bradix,linear", "star"] = True


step2_results

In [ ]:
all_results = pd.concat((step1_results, step2_results)).reset_index(drop=True)
mask = pareto_mask(all_results)
all_results.sort_values("pred_time")

In [ ]:
display(HTML(all_results.sort_values("layers").to_html()))

In [ ]:
def human_size(bytes, units=[' bytes','KB','MB','GB','TB', 'PB', 'EB']):
    """ Returns a human readable string reprentation of bytes"""
    return str(bytes) + units[0] if bytes < 1024 else human_size(bytes>>10, units[1:])

all_hresults = all_results.copy()
all_hresults["size linear search"] = [human_size(x) for x in all_results["size linear search"].to_list()]
all_hresults["size binary search"] = [human_size(x) for x in all_results["size binary search"].to_list()]
all_hresults[mask].sort_values("pred_time")

In [ ]:
ax = all_results.plot.scatter("size binary search", "pred_time", color="orange")
all_results[mask].plot.scatter("size binary search", "pred_time", ax=ax)

In [ ]:
all_results["binary"] = all_results["pred_time_lin"] >= all_results["pred_time_bin"]

final_configs = []
for idx, row in all_results[mask][["layers", "branching factor", "namespace", "binary"]].iterrows():
    final_configs.append(row.to_dict())

                         
final_results = parallel_test_rmis(final_configs, phase="step3")

In [ ]:
def measure_rmis(configs):
    os.system("rm -rf SOSD/build")
    os.system("rm -rf SOSD/competitors/rmi/nm*")
    for config in configs:
        ns = config["namespace"]
        os.system(f"cp opt/{ns}.cpp SOSD/competitors/rmi/")
        os.system(f"cp opt/{ns}_data.h SOSD/competitors/rmi/")
        os.system(f"cp opt/{ns}.h SOSD/competitors/rmi/")
        
    with open("SOSD/benchmark.cc.mustache", "r") as f:
        template = f.read()

    with open("SOSD/benchmark.cc", "w") as f:
        f.write(pystache.render(template, 
                                {"candidates": configs}))

    os.system("cd SOSD && scripts/prepare.sh")
    os.system(f"cd SOSD && build/benchmark {DATA_PATH} {DATA_PATH}_equality_lookups_10M > times.txt")

    times = []
    with open("SOSD/times.txt", "r") as f:
        for l in f:
            if not l.startswith("RESULT"):
                continue
            if "," not in l:
                continue
            result = l.split(",")[1]
            if result == "-1":
                times.append(float("NaN"))
            else:
                times.append(float(result))
    return times

In [ ]:
times = measure_rmis(final_configs)
tested_results = all_results[mask].copy()
tested_results["measured"] = times
tested_results.plot.scatter("pred_time", "measured")

In [ ]:
failed_mask = tested_results["measured"].isna()
tested_results[failed_mask][["layers", "branching factor"]]

In [ ]:
final_mask = pareto_mask(tested_results, v2="measured", soft=False)
tested_results[final_mask].plot.scatter("size linear search", "measured")
tested_results[final_mask].sort_values("measured")

In [ ]:
x = []
y = []
for idx, r in tested_results.iterrows():
    y.append(r.measured)
    
    features = []
    features.append(r["inference"])
    if r.binary:
        features.append(bin_pred.predict([[r["max error"]]])[0])
    else:
        features.append(lin_pred.predict([[r["max error"]]])[0])
    x.append(features)

        
x = np.array(x)
y = np.array(y)

from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x, y)

plt.scatter(tested_results.pred_time, tested_results.measured)
plt.scatter(reg.predict(x), tested_results.measured, color="red")

In [ ]:
x = []
for idx, r in all_results.iterrows():
    features = []
    features.append(r["inference"])
    if r.binary:
        features.append(bin_pred.predict([[r["max error"]]])[0])
    else:
        features.append(lin_pred.predict([[r["max error"]]])[0])
    x.append(features)

all_results["adj pred time"] = reg.predict(x)
new_mask = pareto_mask(all_results, v2="adj pred time")
adj_on_front = all_results[new_mask].copy()

redo_configs = []
adj_mask = []
for adj in adj_on_front.T.to_dict().values():
    res = tested_results[(tested_results.layers == adj["layers"]) & (tested_results["branching factor"] == adj["branching factor"])]
    if len(res) != 1:
        adj_mask.append(True)
        redo_configs.append({
            "layers": adj["layers"],
            "branching factor": adj["branching factor"],
            "namespace": f"nm{nmspc_counter}",
            "binary": adj["max error"] > 100
        })
        nmspc_counter += 1
    else:
        adj_mask.append(False)
        

In [ ]:
if redo_configs:
    print(len(redo_configs), "to re-test after updating inference model")
    parallel_test_rmis(redo_configs, phase="adj")
    new_times = measure_rmis(redo_configs)
    new_results = adj_on_front[adj_mask].copy()
    new_results["measured"] = new_times
    tested_results = pd.concat((new_results, tested_results), sort=False).reset_index(drop=True)

In [ ]:
final_mask = pareto_mask(tested_results, v2="measured", ignore_star=True)
ax = tested_results.plot.scatter("size linear search", "measured", color="red")
tested_results[final_mask].plot.scatter("size linear search", "measured", ax=ax)

In [ ]:
final_results = tested_results[final_mask].copy()
final_results.plot.scatter("size linear search", "measured")
final_results["size"] = final_results["binary"].astype(np.int) * final_results["size binary search"] + ((final_results["binary"].astype(np.int) + 1) % 2) * final_results["size linear search"]
final_results[["layers", "branching factor", "average error", "max error", "size", "measured"]].sort_values("measured")